In [2]:
from csv_loader import tweets, test_tweets
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

import numpy as np

In [3]:
stemmer = PorterStemmer()


def check_base_word(base_word):
	if base_word and len(base_word) > 1 and base_word.isalpha():
		stemmed_word = stemmer.stem(base_word)
		if stemmed_word not in base_vector and stemmed_word not in stopwords.words('english'):
			return True
	return False


# Create base vector
base_vector = []
for tweet in tweets:
	if tweet['text']:
		words = tweet['text'].split()
		for word in words:
			if check_base_word(word):
				base_vector.append(stemmer.stem(word))
                

# Transform tweets into vectors
tweet_vectors = []
category_labels = []
subcategory_labels = []
for tweet in tweets:
    category_labels.append(tweet['category'])
    subcategory_labels.append(tweet['subcategory'])
    words = list(set(tweet['text'].split()))
    stemmed_words = [stemmer.stem(word) for word in words if word and len(word) > 1 and word.isalpha()]
    tweet_vector = []
    for ele in base_vector:
        tweet_vector.append(1 if ele in stemmed_words else 0)
    tweet_vectors.append(tweet_vector)
        
# Transform tweets into vectors
test_tweet_vectors = []
test_category_labels = []
test_subcategory_labels = []
for tweet in test_tweets:
    test_category_labels.append(tweet['category'])
    test_subcategory_labels.append(tweet['subcategory'])
    words = list(set(tweet['text'].split()))
    stemmed_words = [stemmer.stem(word) for word in words if word and len(word) > 1 and word.isalpha()]
    test_tweet_vector = []
    for ele in base_vector:
        test_tweet_vector.append(1 if ele in stemmed_words else 0)
    test_tweet_vectors.append(test_tweet_vector)
        


In [12]:
tensor_tweet_vectors = torch.LongTensor(tweet_vectors)
tensor_category_labels = torch.LongTensor(category_labels)
tensor_subcategory_labels = torch.LongTensor(subcategory_labels)


tensor_test_tweet_vectors = torch.LongTensor(test_tweet_vectors)
tensor_test_category_labels = torch.LongTensor(test_category_labels)
tensor_test_subcategory_labels = torch.LongTensor(test_subcategory_labels)


In [13]:
# Hyper Parameters
input_size = 399
num_classes = 5
num_epochs = 5
batch_size = 100
learning_rate = 0.002

# Model
class LogisticRegression(nn.Module):
	def __init__(self, input_size, num_classes):
		super(LogisticRegression, self).__init__()
		self.linear = nn.Linear(input_size, num_classes)
	
	def forward(self, x):
		out = self.linear(x)
		return out


model = LogisticRegression(input_size, num_classes)

# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training the Model
for epoch in range(num_epochs):
	for i, (tensor_tweet_vector) in enumerate(tensor_tweet_vectors):
		tensor_tweet_vector = Variable(tensor_tweet_vector)
		labels = Variable(tensor_category_labels)
		
		# Forward + Backward + Optimize
		optimizer.zero_grad()
		outputs = model(tensor_tweet_vector)
		loss = criterion(outputs, labels)
		loss.backward()
		optimizer.step()
		
		if (i + 1) % 100 == 0:
			print('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' % (
			epoch + 1, num_epochs, i + 1, len(tensor_tweet_vectors) // batch_size, loss.data[0]))

# Test the Model
correct = 0
total = 0
for tensor_test_tweet_vector in tensor_test_tweet_vectors:
	images = Variable(tensor_test_tweet_vectors)
	outputs = model(tensor_test_tweet_vector)
	_, predicted = torch.max(outputs.data, 1)
	total += labels.size(0)
	correct += (predicted == labels).sum()

print('Accuracy of the model on the 100 test: %d %%' % (100 * correct / total))

# Save the Model
torch.save(model.state_dict(), 'model.pkl')

RuntimeError: Expected object of type Variable[torch.LongTensor] but found type Variable[torch.FloatTensor] for argument #1 'mat2'